In [ ]:
"""
Grid preparation for future aspen project:

Author: maxwell.cook@colorado.edu
"""

import os, sys, time, re
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
import rasterio as rio
import numpy as np

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/')

# Custom functions
sys.path.append(os.path.join(maindir,'aspen-fire/Aim2/code/Python'))
from __functions import *

proj = 'EPSG:5070' # albers

print("Ready to go !")

## Future Fire

From Stephens et al., In review, annual probability of area burned and fire occurrence. Trend to 2060 was calculated using a median-based linear model (MBLM) thielsen estimate. 

In [ ]:
# load the firesheds
fp = os.path.join(projdir,'Aim3/data/spatial/raw/fsim/subfiresheds.gpkg')
firesheds = gpd.read_file(fp)
firesheds.columns

In [ ]:
# tidy the dataframe
firesheds.drop(
    columns=[
        'OBJECTID','Fireshed_State',
        'Shape_Length','Shape_Area',
        'Fireshed_Area_Ha','Fireshed_Code',
        'Fireshed_MajRegion'
    ], inplace=True
)
firesheds.rename(
    columns={
        'Fireshed_ID': 'fs_id',
        'Fireshed_Name': 'fs_name',
        'Subfireshed_ID': 'sfs_id',
        'Subfireshed_Area_Ha': 'sfs_area_ha',
        'AnnualExposure': 'sfs_exposure',
        'PctRecentlyDisturbed': 'pct_disturbed'
    }, inplace=True
)
# check the results
firesheds.head()

### Wildland Urban Interface/Intermix (SILVIS)

In [ ]:
# calculate the zonal stats for the WUI classes
# load the SILVIS Lab 10-m WUI classification
wui_fp = os.path.join(projdir, 'Aim3/data/spatial/raw/silvis/srm_wui_silvis_10m.tif')
wui = rxr.open_rasterio(wui_fp, masked=True, chunks='auto').squeeze()
print(wui)

In [ ]:
# calculate the percent cover of WUI classes for each grid
t0 = time.time()

# see __functions.py
wui_grid = compute_band_stats(firesheds, wui, 'sfs_id', attr='wui')
# tidy columns in the summary table
wui_grid['count'] = wui_grid['count'].astype(int)
wui_grid['total_pixels'] = wui_grid['total_pixels'].astype(int)
wui_grid.rename(columns = {'count': 'wui_pixels'}, inplace=True)

print(wui_grid.head()) # check the results

t1 = (time.time() - t0) / 60
print(f"\nTotal elapsed time: {t1:.2f} minutes.")
print("\n~~~~~~~~~~\n")

In [ ]:
wui_grid['wui'].unique()

In [ ]:
# join the WUI description
# create the mappings
wui_desc = {
    0: 'No Data',
    1: 'Forest/Shrubland/Wetland-dominated Intermix WU',
    2: 'Forest/Shrubland/Wetland-dominated Interface WUI',
    3: 'Grassland-dominated Intermix WUI',
    4: 'Grassland -dominated Interface WUI',
    5: 'Non-WUI: Forest/Shrub/Wetland-dominated',
    6: 'Non-WUI: Grassland-dominated',
    7: 'Non-WUI: Urban',
    8: 'Non-WUI: Other'
}

# join back to the results
wui_grid['wui_desc'] = wui_grid['wui'].map(wui_desc)
wui_grid.head()

In [ ]:
# save this file out
out_fp = os.path.join(projdir,'Aim3/data/tabular/firesheds_wui_summary.csv')
wui_grid.to_csv(out_fp)
print(f"File saved to: {out_fp}")

In [ ]:
# pivot the table to get WUI percent cover
# retain columns for just the interface/intermix classes
wui_w = wui_grid.pivot_table(index='sfs_id', columns='wui', values='pct_cover', fill_value=0)
wui_w = wui_w[[1, 2, 3, 4]] # keep only WUI classes
wui_w.columns = [f"wui{int(col)}" for col in wui_w.columns] # rename the columns
wui_w = wui_w.reset_index()
wui_w.head()

In [ ]:
# reclassify into a binary raster (WUI / Non-WUI)
wui_bin = xr.where(wui.isin([1,2,3,4]), 1, 0)
wui_bin = wui_bin.assign_attrs(wui.attrs) # assign the array attributes
# save this raster out
out_path = os.path.join(projdir, 'Aim3/data/spatial/mod/silvis/srm_wui_silvis_10m_bin.tif')
wui_bin.rio.to_raster(
    out_path,
    compress='zstd', zstd_level=9,
    dtype=rio.uint8, driver='GTiff'
)
print(f"Saved file to: {out_path}")

In [ ]:
# resample to a coarser resolution to compute the euclidean distance

import subprocess
from osgeo import gdal

# define a function for the gdal warp implementation
def resample_grid_gdalwarp(in_file, out_file, extent=None, resample_method='max', res=90):
    """
    Resample a raster file from 10-meter to 90-meter resolution using gdalwarp.

    :param in_file: The input raster file path.
    :param out_file: The output raster file path.
    :param resampling_method: The resampling method to use ('sum', 'average', etc.).
    :param target_resolution: The target resolution in meters (default is 250).
    :return: None
    """
    try:
        # Construct the gdalwarp command
        command = [
            'gdalwarp',
            '-tr', str(res), str(res),  # Target resolution
            '-r', resample_method,  # Resampling method
            '-of', 'GTiff',  # Output format
            '-co', 'COMPRESS=LZW',  # Compression
            '-ot', 'Int8', # ensure output data type
            '-t_srs', 'EPSG:5070', # output CRS
            in_file,
            out_file
        ]

        # Run the command
        subprocess.run(command, check=True)
        print(f'Successfully resampled {in_file} to {out_file}')

    except subprocess.CalledProcessError as e:
        print(f"Error resampling {in_file}: {e}")

# apply the function
in_path = os.path.join(projdir, 'Aim3/data/spatial/mod/silvis/srm_wui_silvis_10m_bin.tif')
out_file = os.path.join(projdir, 'Aim3/data/spatial/mod/silvis/srm_wui_silvis_10m_bin_90m.tif')
resample_grid_gdalwarp(in_path, out_file) # run the process

In [ ]:
# calculate the euclidean distance array
from scipy.ndimage import distance_transform_edt
# load the 250m resampled grid created above
wui_90 = rxr.open_rasterio(out_file, masked=True).squeeze()

t0 = time.time()
# run the euclidean distance
distance_to_wui = distance_transform_edt(wui_90 == 0)
# convert to an xarray data array
distance_xr = xr.DataArray(distance_to_wui, coords=wui_90.coords, dims=wui_90.dims, attrs=wui_90.attrs)

# export the result
distance_fp = os.path.join(projdir, 'Aim3/data/spatial/mod/silvis/distance_to_wui.tif')
distance_xr.rio.to_raster(
    distance_fp,
    compress="zstd", zstd_level=9,  # Efficient compression
    dtype="float32", driver="GTiff"
)

t1 = (time.time() - t0) / 60
print(f"\nTotal elapsed time: {t1:.2f} minutes.")
print("\n~~~~~~~~~~\n")

In [ ]:
# calculate the average distance to WUI for gridcells

t0 = time.time()

# run the zonal stats
zs = compute_band_stats(
    geoms=firesheds, 
    image_da=distance_xr, 
    id_col='sfs_id', 
    stats=['mean'], # 'median','std','percentile_90'
    attr='wui_dist',
    ztype='continuous'
)

# check the results:
print(zs.head())

t1 = (time.time() - t0) / 60
print(f"\nTotal elapsed time: {t1:.2f} minutes.")
print("\n~~~~~~~~~~\n")

In [ ]:
# merge the WUI summary stats by gridcell
wui_grid_stats = pd.merge(wui_w, zs, on='sfs_id', how='right')
wui_grid_stats.head()

In [ ]:
print(len(firesheds))
print(len(wui_grid_stats))

In [ ]:
wui_grid_stats.isna().sum()

In [ ]:
# export the summary table
out_fp = os.path.join(projdir, 'Aim3/data/tabular/fireshed_wuiSILVIS_stats.csv')
wui_grid_stats.to_csv(out_fp)
print(f"Saved to: {out_fp}")